In [1]:
import os
base_dir = 'C:\\Users\heine\github\PlateReaderTools'  #You will need to change this depending on which computer you are working on
os.chdir(base_dir) 

#import sys
from core import plate_reader_tools 
import pandas as pd
import numpy as np
# import math
# import matplotlib
import matplotlib.pyplot as plt
import string
import seaborn as sns
from itertools import chain

# import os
# import sys

# path = '/Users/andrewng/Documents/PlateReaderTools'
# if not path in sys.path:
#     sys.path.insert(1, path)
# del path

# from plate_reader_tools import *
# import seaborn as sns
# sns.set_style("white")
# sns.set_style("ticks")

%load_ext autoreload
%autoreload 2
%matplotlib notebook
#%matplotlib inline

In [7]:
dirname = base_dir + os.path.normpath("/scripts/templates") + os.sep
fname = dirname + "20170812_HIS_integration_growth_20170316_BMH_24h.xlsx"
#date = fname[-13:-5]
sheetname = 'Result sheet'
skiprows = 50
rows = 8
columns = 12
datalabels = ['OD','YFP'] 
cycles = 88
horz = 0


In [8]:
[d,time_list] = plate_reader_tools.readplate(fname,sheetname, skiprows, rows, columns, datalabels, cycles, horz)
OD_data = d['OD']
OD_data

In [ ]:
OD_data_filtered, OVER_list = plate_reader_tools.over_filter(OD_data)

In [ ]:
#Plot all wells in an array similar to the plate. 
letter_range = ['A','B','C','D','E','F','G','H']
number_range = range(1,13)
well_arr = plate_reader_tools.well_array_builder(letter_range,number_range)
fig = plate_reader_tools.subplot_array(OD_data.replace('OVER',np.nan), well_arr, time_list)

In [ ]:
#Build Multiindex dataframe to compare different conditions.  
layer_names = ['Media','Strain','Background','Tech_Rep']
backgrounds = ['w303','BY4741']
media = ['YPD','SD-HIS','SDC']
strains = ['+YTK190 1', '+YTK190 2', '+YTK190 3', '+pTMP136 1', '+pTMP136 2', '+pTMP136 3', 'AHN321', 'HES5-41','WCD230','yBMH126', 'yBMH127']
tech_reps = ['TR1','TR2']      
experiment_setup = [media, strains, backgrounds, tech_reps]      
data_index = pd.MultiIndex.from_product(experiment_setup, names= layer_names)
print(data_index.levels)
print(data_index.names)
         


In [ ]:
#remove missing conditions from data
# Step 1: make index list for the coordinates of all the missing items based on pairs of missing conditions. 
 
# #w303 background doesn't have 'WCD230','AHN321', 'yBMH126', 'yBMH127' strains. 
# 'HES5-41','yBMH126', 'yBMH127' don't have tech rep 2  
missing_items = [('Background','w303','Strain','yBMH126'),
                 ('Background','w303','Strain','yBMH127'),
                 ('Background','w303','Strain','WCD230'),
                 ('Background','w303','Strain','AHN321'),
                 ('Background','BY4741','Strain','HES5-41'),
                 ('Strain','yBMH126','Tech_Rep','TR2'),
                 ('Strain','yBMH127','Tech_Rep','TR2'),
                 ('Strain','HES5-41','Tech_Rep','TR2')
                ]

data_index_adjusted, inds_to_remove = plate_reader_tools.delete_multiindex_for_missing_conditions(missing_items, data_index)

#Inds to remove should be constructed from this list of lists: 
#[[36, 37, 80, 81, 124, 125], [40, 41, 84, 85, 128, 129], [32, 33, 76, 77, 120, 121], [24, 25, 68, 69, 112, 113], [30, 31, 74, 75, 118, 119], [37, 39, 81, 83, 125, 127], [41, 43, 85, 87, 129, 131], [29, 31, 73, 75, 117, 119]]

In [ ]:
#Step 2 assign wells in the correct order to the index.  This is a manuyal process - you either can usethis tool
#to print out the order of the conditions so that you can generate a list of wells yourself or use
#the input function to build it one well at a time.  
#If you break in the middle of the routine the well_list will be there in memory or you can copy / paste it from the screen

well_list = plate_reader_tools.view_or_input_exp_design(data_index_adjusted)



print out list (1) or add wells one by one (2)2
input well for ['YPD', '+YTK190 1', 'w303', 'TR1'] : F1
correct well: F1? 1=Yes, 0 = No, exit = break loop1
['F1']
input well for ['YPD', '+YTK190 1', 'w303', 'TR2'] : F7
correct well: F7? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7']
input well for ['YPD', '+YTK190 1', 'BY4741', 'TR1'] : C1
correct well: C1? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1']
input well for ['YPD', '+YTK190 1', 'BY4741', 'TR2'] : C7
correct well: C7? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7']
input well for ['YPD', '+YTK190 2', 'w303', 'TR1'] : G1
correct well: G1? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1']
input well for ['YPD', '+YTK190 2', 'w303', 'TR2'] : G7
correct well: G7? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7']
input well for ['YPD', '+YTK190 2', 'BY4741', 'TR1'] : D1
correct well: D1? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1']
input well for ['YPD', '+YTK190 2', 'BY4741', 'TR2'] : D7
correct well: D7? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7']
input well for ['YPD', '+YTK190 3', 'w303', 'TR1'] : H1
correct well: H1? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1']
input well for ['YPD', '+YTK190 3', 'w303', 'TR2'] : H7
correct well: H7? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7']
input well for ['YPD', '+YTK190 3', 'BY4741', 'TR1'] : E1
correct well: E1? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1']
input well for ['YPD', '+YTK190 3', 'BY4741', 'TR2'] : E7
correct well: E7? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7']
input well for ['YPD', '+pTMP136 1', 'w303', 'TR1'] : F2
correct well: F2? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2']
input well for ['YPD', '+pTMP136 1', 'w303', 'TR2'] : F8
correct well: F8? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8']
input well for ['YPD', '+pTMP136 1', 'BY4741', 'TR1'] : C2
correct well: C2? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2']
input well for ['YPD', '+pTMP136 1', 'BY4741', 'TR2'] : C8
correct well: C8? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8']
input well for ['YPD', '+pTMP136 2', 'w303', 'TR1'] : G2
correct well: G2? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2']
input well for ['YPD', '+pTMP136 2', 'w303', 'TR2'] : G8
correct well: G8? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8']
input well for ['YPD', '+pTMP136 2', 'BY4741', 'TR1'] : D2
correct well: D2? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2']
input well for ['YPD', '+pTMP136 2', 'BY4741', 'TR2'] : D8
correct well: D8? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8']
input well for ['YPD', '+pTMP136 3', 'w303', 'TR1'] : H2
correct well: H2? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8', 'H2']
input well for ['YPD', '+pTMP136 3', 'w303', 'TR2'] : H8
correct well: H8? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8', 'H2', 'H8']
input well for ['YPD', '+pTMP136 3', 'BY4741', 'TR1'] : E2
correct well: E2? 1=Yes, 0 = No, exit = break loopE8
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8', 'H2', 'H8', 'E2']
input well for ['YPD', '+pTMP136 3', 'BY4741', 'TR2'] : E8
correct well: E8? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8', 'H2', 'H8', 'E2', 'E8']
input well for ['YPD', 'AHN321', 'BY4741', 'TR1'] : B1
correct well: B1? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8', 'H2', 'H8', 'E2', 'E8', 'B1']
input well for ['YPD', 'AHN321', 'BY4741', 'TR2'] : B7
correct well: B7? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8', 'H2', 'H8', 'E2', 'E8', 'B1', 'B7']
input well for ['YPD', 'HES5-41', 'w303', 'TR1'] : A7
correct well: A7? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8', 'H2', 'H8', 'E2', 'E8', 'B1', 'B7', 'A7']
input well for ['YPD', 'WCD230', 'BY4741', 'TR1'] : B2
correct well: B2? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8', 'H2', 'H8', 'E2', 'E8', 'B1', 'B7', 'A7', 'B2']
input well for ['YPD', 'WCD230', 'BY4741', 'TR2'] : B8
correct well: B8? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8', 'H2', 'H8', 'E2', 'E8', 'B1', 'B7', 'A7', 'B2', 'B8']
input well for ['YPD', 'yBMH126', 'BY4741', 'TR1'] : A1
correct well: A1? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8', 'H2', 'H8', 'E2', 'E8', 'B1', 'B7', 'A7', 'B2', 'B8', 'A1']
input well for ['YPD', 'yBMH127', 'BY4741', 'TR1'] : A2
correct well: A2? 1=Yes, 0 = No, exit = break loop1
['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8', 'H2', 'H8', 'E2', 'E8', 'B1', 'B7', 'A7', 'B2', 'B8', 'A1', 'A2']
input well for ['SD-HIS', '+YTK190 1', 'w303', 'TR1'] : asdfas
correct well: asdfas? 1=Yes, 0 = No, exit = break loopexit
exiting loop


25, 26 (231)
30, 31 (126, 127)

In [ ]:
#The YPD list was generated with the routine above.  The SDC lists were then generated from the YPD list
#manually typing in coordinates.  This will be customized for each experiment layout.  

well_list = {'YPD' : ['F1', 'F7', 'C1', 'C7', 'G1', 'G7', 'D1', 'D7', 'H1', 'H7', 
                      'E1', 'E7', 'F2', 'F8', 'C2', 'C8', 'G2', 'G8', 'D2', 'D8', 
                      'H2', 'H8', 'E2', 'E8', 'B1', 'B7', 'A7', 'B2', 'B8', 'A1',
                      'A2'], 
            'SD-HIS' : ['F3', 'F9', 'C3', 'C9', 'G3', 'G9', 'D3', 'D9', 'H3', 'H9', 'E3', 'E9',
                        'F4', 'F10', 'C4', 'C10', 'G4', 'G10', 'D4', 'D10', 'H4', 'H10', 'E4', 'E10',
                        'B3', 'B9', 'A9', 'B4', 'B10', 'A3', 'A4'], 
             'SDC' : ['F5', 'F11', 'C5', 'C11', 'G5', 'G11', 'D5', 'D11', 'H5', 'H11', 'E5', 'E11',
                    'F6', 'F12', 'C6', 'C12', 'G6', 'G12', 'D6', 'D12', 'H6', 'H12', 'E6', 'E12',
                    'B5', 'B11', 'A11', 'B6', 'B11', 'A5', 'A6']}


In [ ]:
blank = {'YPD':np.mean(OD_data['A8']),
        'SD-HIS':np.mean(OD_data['A10']),
        'SDC':np.mean(OD_data['A12'])}

growth_data = []
for medium in media: 
    growth_data.append([OD_data[well]-blank[medium] for well in well_list[medium] ])
growth_data = list(chain.from_iterable(growth_data))

                          
                          
growth_data_df = pd.DataFrame(growth_data, index=data_index_adjusted)

#make columns time points
dt = 15.0/60.0
growth_data_df.columns = growth_data_df.columns*dt

In [ ]:
#Plot raw growth curves for all conditions

fig, ax = plt.subplots(2, 3,sharex = True, sharey = True)

for jj, medium in enumerate(media): 
    for kk, background in enumerate(backgrounds):
        #Select data with correct medium and background
        growth_data_df_med_bg = growth_data_df.xs((medium,background),level = ['Media','Background'])
        
        #average across technical replicates.
        growth_data_df_med_bg_avg = growth_data_df_med_bg.mean(level='Strain')

        #Plot growth curves
        #only show labels on right side of the plot. 
        if jj ==2: 
            growth_data_df_med_bg_avg.transpose().plot(ax = ax[kk,jj], title = medium + ' ' + background, legend = True)
            ax[kk,jj].legend(loc='center left', bbox_to_anchor=(1, 0.5))
        else: 
            growth_data_df_med_bg_avg.transpose().plot(ax = ax[kk,jj], title = medium + ' ' + background, legend = False)




In [ ]:
#Plot log2 growth curves for all conditions
fig, ax = plt.subplots(2, 3,sharex = True, sharey = True)

for jj, medium in enumerate(media): 
    for kk, background in enumerate(backgrounds):
        #Select data with correct medium and background
        growth_data_df_med_bg = growth_data_df.xs((medium,background),level = ['Media','Background'])
        
        #average across technical replicates.
        growth_data_df_med_bg_avg = growth_data_df_med_bg.mean(level='Strain')
        
        #take Log(base2) of data. 
        growth_data_df_med_bg_avg_log = np.log(growth_data_df_med_bg_avg)/np.log(2)
        
        #Plot growth curves
        #only show labels on right side of the plot. 
        if jj ==2: 
            growth_data_df_med_bg_avg_log.transpose().plot(ax = ax[kk,jj], title = medium + ' ' + background, legend = True)
            ax[kk,jj].legend(loc='center left', bbox_to_anchor=(1, 0.5))
        else: 
            growth_data_df_med_bg_avg_log.transpose().plot(ax = ax[kk,jj], title = medium + ' ' + background, legend = False)




Looking at the log plot the linear growth range is between about 2h and 7h.  I'll just use YPD for the next plot which will be a bar plot of growth rates. 

In [ ]:
#This actually only seems true for BY4741.  For W303, the linear range looks more like 0 to 2


In [ ]:
#plot log of YPD BY4741 strains in linear range

#In between time 1 and 5 log plot looked linear.  
t = growth_data_df_med_bg_avg_log.columns
t_low = 1.0 
t_high = 5.0
t_check = (t>t_low)*(t<t_high)
#finds indices of all time values above t_low
t_inds = [ind for ind,val in enumerate(t_check) if val==True]

t_linear_range = t[t_inds]

#Select YPD media and BY4741 background
media = 'YPD'
background = 'BY4741'
growth_data_df_med_bg = growth_data_df.xs((media,background),level = ['Media','Background'])

#Take average
growth_data_df_med_bg_avg = growth_data_df_med_bg.mean(level='Strain')

#Take Log2
growth_data_df_med_bg_avg_log = np.log(growth_data_df_med_bg_avg)/np.log(2)

#Extract only linear range
growth_data_df_med_bg_avg_log_linrange = growth_data_df_med_bg_avg_log.iloc[:,t_inds]

#Plot
fix, ax = plt.subplots()
growth_data_df_med_bg_avg_log_linrange.transpose().plot(ax = ax, title = media + ' ' + background, legend = True)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    

In [ ]:
#Make Barplot of slopes: 

#Look for good sns method for plotting a barplot of the different strains.  
fig, ax = plt.subplots()
growth_rates_YPD_BY4741 = pd.DataFrame(growth_data_df_med_bg_avg_log_linrange.T.apply(plate_reader_tools.get_slope))
growth_rates_YPD_BY4741["new_labels"] = ["New His 1", "New His 2", "New His 3", "Old His 1", "Old His 2", "Old His 3", "WT+dCas9","WT","Old strain no deg","Old strain deg"]
growth_rates_YPD_BY4741.set_index("new_labels",inplace=True)
growth_rates_YPD_BY4741.plot(kind='bar', ax=ax, rot = 45, legend = False,)
ax.set_ylabel("Growth rate (doublings/hr)")


In [ ]:
#plot log of YPD W303 strains in linear range

#In between time 0 and 1.5 log plot looked linear.  
t = growth_data_df_med_bg_avg_log.columns
t_low = 0.0 
t_high = 1.5
t_check = (t>t_low)*(t<t_high)
#finds indices of all time values above t_low
t_inds = [ind for ind,val in enumerate(t_check) if val==True]

t_linear_range = t[t_inds]

#Select YPD media and BY4741 background
media = 'YPD'
background = 'w303'
growth_data_df_med_bg = growth_data_df.xs((media,background),level = ['Media','Background'])

#Take average
growth_data_df_med_bg_avg = growth_data_df_med_bg.mean(level='Strain')

#Take Log2
growth_data_df_med_bg_avg_log = np.log(growth_data_df_med_bg_avg)/np.log(2)

#Extract only linear range
growth_data_df_med_bg_avg_log_linrange = growth_data_df_med_bg_avg_log.iloc[:,t_inds]

#Plot
fix, ax = plt.subplots()
growth_data_df_med_bg_avg_log_linrange.transpose().plot(ax = ax, title = media + ' ' + background, legend = True)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
#Make Barplot of slopes: 

#Look for good sns method for plotting a barplot of the different strains.  
fig, ax = plt.subplots()
growth_rates_YPD_W303 = pd.DataFrame(growth_data_df_med_bg_avg_log_linrange.T.apply(plate_reader_tools.get_slope))
growth_rates_YPD_W303["new_labels"] = ["New His 1", "New His 2", "New His 3", "Old His 1", "Old His 2", "Old His 3", "WT"]
growth_rates_YPD_W303.set_index("new_labels",inplace=True)
growth_rates_YPD_W303.plot(kind='bar', ax=ax, rot = 45, legend = False,)
ax.set_ylabel("Growth rate (doublings/hr)")
#fig.savefig(dirname + "plots//YPD_w303_growth_rate.png")